In [2]:
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
# https://towardsdatascience.com/how-to-create-and-customize-venn-diagrams-in-python-263555527305
from matplotlib_venn import venn2, venn2_circles
from matplotlib_venn import venn3, venn3_circles

import numpy as np

from extract.jobs import parse_mallob, get_baseline

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [3]:
static_2_df = pd.DataFrame(parse_mallob("benchmark/static_5_1_2_random"))
static_4_df = pd.DataFrame(parse_mallob("benchmark/static_4_1_4_new"))
static_8_df = pd.DataFrame(parse_mallob("benchmark/static_3_1_8_random"))

In [14]:
baseline_df = pd.DataFrame(get_baseline("benchmark/baseline.txt"))

TypeError: get_baseline() takes 0 positional arguments but 1 was given

In [4]:
two = static_2_df.set_index("identifier")["result"].rename("two")
four = static_4_df.set_index("identifier")["result"].rename("four")
eight = static_8_df.set_index("identifier")["result"].rename("eight")

result = pd.concat([two, four, eight], axis=1, join="outer").fillna("UNKNOWN")

result.groupby(['two', 'four', 'eight']).size().reset_index().rename(columns={0:'count'})

,two,four,eight,count
0,SAT,SAT,SAT,90
1,SAT,SAT,UNKNOWN,9
2,SAT,UNKNOWN,SAT,5
3,SAT,UNKNOWN,UNKNOWN,10
4,UNKNOWN,SAT,SAT,2
5,UNKNOWN,SAT,UNKNOWN,3
6,UNKNOWN,UNKNOWN,SAT,9
7,UNKNOWN,UNKNOWN,UNKNOWN,196
8,UNKNOWN,UNKNOWN,UNSAT,6
9,UNKNOWN,UNSAT,UNKNOWN,1


In [9]:
fig = plt.figure()

# Schnittmenge der gelösten (Löst es die gleichen Instanzen?)
venn3(subsets = (10 + 3, 3 + 1, 9 + 1, 9 + 6, 5 + 1, 2 + 3, 90 + 40), set_labels =('$J=2$', '$J=4$', '$J=8$'), set_colors=('#377eb8', '#ff7f00', '#e41a1c'))

fig.set_size_inches(w=3, h=2)
fig.savefig("scale/venn.pdf")

In [13]:
# Vergleiche nicht random mit random
fig = plt.figure()
fig.subplots_adjust(bottom=0.2, left=0.175)

temp_series = static_2_df[static_2_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
ax = temp_series.plot(color="#377eb8", style="-.", figsize=(3,2))

temp_series = static_4_df[static_4_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#ff7f00", style="--", ax=ax)

temp_series = static_8_df[static_8_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#e41a1c", style=":", ax=ax)

ax.set_xlabel("run time in s")
ax.set_ylabel("\# solved problems in $\leq t$ s")

ax.legend(['$J=2$', '$J=4$', '$J=8$'])

fig.savefig("scale/time.pdf")

In [8]:
fig = plt.figure()
fig.subplots_adjust(bottom=0.2, left=0.175)
temp_series = static_2_df[static_2_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
ax = temp_series.plot(color="r", figsize=(3,2))

temp_series = static_4_df[static_4_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="g", ax=ax)

temp_series = static_8_df[static_8_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="b", ax=ax)

ax.set_xlabel("run time in s")
ax.set_ylabel("generated cubes in $\leq t$ s")

ax.legend(['$J=2$', '$J=4$', '$J=8$'])

fig.savefig("scale/generated.pdf")

# Passende caption hier wichtig
# Cubes generated for number of problems in the given time.